In [2]:
import pandas as pd

In [4]:
df = pd.read_csv("../data/step1_clean.csv")

In [5]:
df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF,temp_diff
0,1,M14860,M,298.1,308.6,1551,42.8,0,0,0,0,0,0,0,10.5
1,2,L47181,L,298.2,308.7,1408,46.3,3,0,0,0,0,0,0,10.5
2,3,L47182,L,298.1,308.5,1498,49.4,5,0,0,0,0,0,0,10.4
3,4,L47183,L,298.2,308.6,1433,39.5,7,0,0,0,0,0,0,10.4
4,5,L47184,L,298.2,308.7,1408,40.0,9,0,0,0,0,0,0,10.5


In [6]:
df = df.drop(columns=['UDI', 'Product ID'])

In [7]:
df = pd.get_dummies(df, columns=['Type'], drop_first=True)
df.head()

,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Tool wear [min],Machine failure,TWF,HDF,PWF,OSF,RNF,temp_diff,Type_L,Type_M
0,298.1,308.6,1551,42.8,0,0,0,0,0,0,0,10.5,False,True
1,298.2,308.7,1408,46.3,3,0,0,0,0,0,0,10.5,True,False
2,298.1,308.5,1498,49.4,5,0,0,0,0,0,0,10.4,True,False
3,298.2,308.6,1433,39.5,7,0,0,0,0,0,0,10.4,True,False
4,298.2,308.7,1408,40.0,9,0,0,0,0,0,0,10.5,True,False


In [10]:
df = df.drop(columns=['TWF','HDF','PWF','OSF','RNF'], errors='ignore')

In [11]:
print(df.columns.tolist())

['Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Tool wear [min]', 'Machine failure', 'temp_diff', 'Type_L', 'Type_M']


In [12]:
X = df.drop('Machine failure', axis=1)
y = df['Machine failure']

print(X.shape, y.shape)

(10000, 8) (10000,)


In [14]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X, y,
    test_size=0.2,
    random_state=42,
    stratify=y
)

print("Train failures:", y_train.mean())
print("Test failures:", y_test.mean())

Train failures: 0.033875
Test failures: 0.034


In [15]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [16]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

classes = np.unique(y_train)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train)

class_weights = dict(zip(classes, weights))
class_weights

{np.int64(0): np.float64(0.51753137533963),
 np.int64(1): np.float64(14.760147601476016)}

In [17]:
import joblib

joblib.dump({
    "X_train": X_train,
    "X_test": X_test,
    "X_train_scaled": X_train_scaled,
    "X_test_scaled": X_test_scaled,
    "y_train": y_train,
    "y_test": y_test,
    "scaler": scaler,
    "class_weights": class_weights
}, "../data/train_test.pkl")

['../data/train_test.pkl']